### 머신 러닝

- 어떤 알고리즘을 사용할 것인가를 판단하기 위해서는 사전 분석을 잘하는 것이 중요합니다.

### 고객의 전체 모습을 파악

- 머신러닝을 하기 위한 데이터 가공 기술을 배우면서 고객 행동을 분석하고 파악

고객의 소리) 
제가 운영하는 스포츠 센터는 트레이닝 붐 덕분에 지금까지 고객 수가 늘었습니다. 그런데 최근 1년간 고객 수가 늘지 않는 것 같습니다. 자주 이용하는 고객은 계속 오지만 가끔 오는 고객은 어느새 오지 않는 경우도 생기는 것 같습니다. 제대로 데이터 분석한 적이 없어서 어떤 고객이 계속 이용하고 있는지조차 모릅니다.             

전제 조건)                
회원 구분 : 종일 회원(센터를 언제든 사용 가능), 주간 회원(낮에만 사용 가능), 야간 회원(밤에만 사용 가능)           
입회비 : 비정기적으로 입회비 반액 할인 / 입회비 무료 행사를 해서 신규 회원을 늘림                 
탈퇴 : 월말까지 신청하면 그 다음 달 말에 탈퇴                     
 
use_log.csv : 센터 이용 이력(회원이 센터를 이용하면 시스템에 자동 입력 - 2018년 4월 ~ 2019년 3월의 1년분 데이터)     
customer_master.csv : 2019년 3월 말 시점의 회원 데이터(이전에 탈퇴한 회원 포함 - 전체 기간)             
class_master.csv : 회원 구분 데이터(종일, 주간, 야간)           
campaign_master.csv : 가입 시 행사 종류 데이터(입회비 유무 등)               

In [2]:
import pandas as pd 
import numpy as np 

In [ ]:
use_log_df = pd.read_csv('./data/1장/use_log.csv') # 이용 기록
len(use_log_df) # 197,428
use_log_df.head()

,log_id,customer_id,usedate
0,L00000049012330,AS009373,2018-04-01
1,L00000049012331,AS015315,2018-04-01
2,L00000049012332,AS040841,2018-04-01
3,L00000049012333,AS046594,2018-04-01
4,L00000049012334,AS073285,2018-04-01


In [ ]:
customer_master_df = pd.read_csv('./data/1장/customer_master.csv') # 고객 데이터
len(customer_master_df) # 4192
customer_master_df

# is_deleted : 201903 시점에 이미 탈퇴한 유저를 시스템에서 빨리 찾기 위한 칼럼
# 회원 데이터는 4,192명으로 이미 탈퇴한 유저도 포함되어 있음
# 3월말에 신청해서 4월에 탈퇴하는 경우 end_date : 0430 & is_delected : 0

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted
0,OA832399,XXXX,C01,F,2015-05-01 00:00:00,NaN,CA1,0
1,PL270116,XXXXX,C01,M,2015-05-01 00:00:00,NaN,CA1,0
2,OA974876,XXXXX,C01,M,2015-05-01 00:00:00,NaN,CA1,0
3,HD024127,XXXXX,C01,F,2015-05-01 00:00:00,NaN,CA1,0
4,HD661448,XXXXX,C03,F,2015-05-01 00:00:00,NaN,CA1,0
...,...,...,...,...,...,...,...,...
4187,HD676663,XXXX,C01,M,2019-03-14 00:00:00,NaN,CA1,0
4188,HD246549,XXXXX,C01,F,2019-03-14 00:00:00,NaN,CA1,0
4189,GD037007,XXXXX,C03,M,2019-03-14 00:00:00,NaN,CA1,0
4190,OA953150,XXXXX,C01,M,2019-03-14 00:00:00,NaN,CA1,0


In [10]:
class_master_df = pd.read_csv('./data/1장/class_master.csv') # 고객 구분(종일/주간/야간)
len(class_master_df) # 3
class_master_df.head()

,class,class_name,price
0,C01,0_종일,10500
1,C02,1_주간,7500
2,C03,2_야간,6000


In [14]:
campaign_master_df = pd.read_csv('./data/1장/campaign_master.csv') # 고객 구분(종일/주간/야간)
len(campaign_master_df) # 3
campaign_master_df.head()

,campaign_id,campaign_name
0,CA1,2_일반
1,CA2,0_입회비반액할인
2,CA3,1_입회비무료


- customer_master의 customer_id + use_log의 customer_id
- customer_master의 class + class_master의 class
- customer_master의 campaign_id + campaign_master의 campaign_id              
-> 결합 가능

>> 분석의 목적이 되는 기준 데이터

이 장의 경우 가능한 것은 고객 데이터인 custeomer_master와 이용 이력 데이터인 use_log입니다.

먼저, 데이터 수가 적은 고객 데이터(customer_master)를 메인으로 진행             
후반에서는 이용 이력 데이터(use_log)를 중심으로 분석도 진행 예정             
일단 이용 이력 데이터는 무시하고, 고객 데이터를 수정해서 <B><U>어떤 고객이 몇 명이었는지</U></B>와 같은 전체 모습을 파악

In [19]:
customer_class_df = pd.merge(left= customer_master_df, right= class_master_df,
                             how= 'left', left_on= 'class', right_on= 'class')
customer_class_df 


customer_merge_df = pd.merge(left= customer_class_df, right= campaign_master_df, how= 'left',
                             on= 'campaign_id')
customer_merge_df # 회원수 4192 그대로 열 3개 추가

# column명 동일하면 left_on, right_on 대신 on 사용
# left_on, right_on 사용하는 경우 두 열 모두 df에 포함되므로 둘 중 한 열은 삭제


,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name
0,OA832399,XXXX,C01,F,2015-05-01 00:00:00,NaN,CA1,0,0_종일,10500,2_일반
1,PL270116,XXXXX,C01,M,2015-05-01 00:00:00,NaN,CA1,0,0_종일,10500,2_일반
2,OA974876,XXXXX,C01,M,2015-05-01 00:00:00,NaN,CA1,0,0_종일,10500,2_일반
3,HD024127,XXXXX,C01,F,2015-05-01 00:00:00,NaN,CA1,0,0_종일,10500,2_일반
4,HD661448,XXXXX,C03,F,2015-05-01 00:00:00,NaN,CA1,0,2_야간,6000,2_일반
...,...,...,...,...,...,...,...,...,...,...,...
4187,HD676663,XXXX,C01,M,2019-03-14 00:00:00,NaN,CA1,0,0_종일,10500,2_일반
4188,HD246549,XXXXX,C01,F,2019-03-14 00:00:00,NaN,CA1,0,0_종일,10500,2_일반
4189,GD037007,XXXXX,C03,M,2019-03-14 00:00:00,NaN,CA1,0,2_야간,6000,2_일반
4190,OA953150,XXXXX,C01,M,2019-03-14 00:00:00,NaN,CA1,0,0_종일,10500,2_일반


>> 조인 후 결측치 확인

In [ ]:
# 조인할 때 키가 없거나 잘못 조인되면 자동으로 결측치가 들어감 > 확인 필요
customer_merge_df.isnull().sum(axis= 0)
customer_merge_df.isnull().sum()

# end_date 외에는 결측치가 0으로, 
# 이번 조인에서 추가한 class_name, price, campaign_name에는 데이터가 정확하게 들어 있는 것을 확인

customer_id         0
name                0
class               0
gender              0
start_date          0
end_date         2842
campaign_id         0
is_deleted          0
class_name          0
price               0
campaign_name       0
dtype: int64

>> 고객 데이터 집계

- 고객 데이터 집계를 통해 고객 전체 모습을 확인
1. 어떤 회원과 어떤 캠페인이 많은지
2. 언제 입회/탈퇴가 많은지
3. 남녀 비율은 얼마인지
4. 탈퇴할 때까지의 기간은 어느 정도인지

In [ ]:
# 회원 class 구분, 캠페인 구분, 성별, 탈퇴 여부 집계
customer_merge_df.groupby(by= 'class_name')['customer_id'].count() # 종일 : 2045, 주간 : 1019, 야간 : 1128

customer_merge_df.groupby(by= 'campaign_name')['customer_id'].count() # 반액 : 650, 무료 : 492, 일반 : 3050

customer_merge_df.groupby(by= 'gender')['customer_id'].count() # 여성 : 1983, 남성 : 2209

customer_merge_df.groupby(by= 'is_deleted')['customer_id'].count() # 고객 : 2842, 탈퇴 : 1350

# 회원 클래스 종일(절반) > 야간 > 주간
# 캠페인 구분 : 일반 입회 > 캠페인 입회 & 캠페인 입회에 의한 가입이 약 20%
# 남자 > 여자 (근소한 차이)
# 2019년 03월 기준) 가입 회원 : 2842명, 탈퇴 회원 : 1350명

# >> 캠페인이 시작된 시기, 성별과 회원 클래스의 관계, 올해 가입 인원 등 궁금증 발생
# 이는 집계해서 확인하기보다 현장 사람들이 아는 경우가 많음

is_deleted
0    2842
1    1350
Name: customer_id, dtype: int64

In [ ]:
# start_date가 2018년 4월 1일 이후부터 2019년 03월 31일까지 가입 인원 집계

# str, float 구분 필요 x
customer_merge_df['start_date'] = pd.to_datetime(customer_merge_df['start_date'])

customer_merge_df['start_date'] # datetime

0      2015-05-01
1      2015-05-01
2      2015-05-01
3      2015-05-01
4      2015-05-01
          ...    
4187   2019-03-14
4188   2019-03-14
4189   2019-03-14
4190   2019-03-14
4191   2019-03-15
Name: start_date, Length: 4192, dtype: datetime64[ns]

In [ ]:
customer_merge_df.loc[(customer_merge_df['start_date'] > '2018-04-01')
                      & (customer_merge_df['start_date'] < '2019-03-31'),
                      'customer_id'].count()

# 책)
lately_1year_customer = customer_merge_df.loc[(customer_merge_df['start_date'] > pd.to_datetime('20180401')),
                      'customer_id']
print(len(lately_1year_customer))

# 해당 기간 동안의 가입 인원 : 1361

1361


>> 최근 고객 정보(3월 '유지' 회원)

- <u>가장 최근 월(2019년 3월)에 회원인 고객</u> 데이터 파악           
현재 고객 데이터에서는 이미 탈퇴한 고객도 포함되어 있어 월별 집계와는 차이가 있습니다.             
<u>가장 최근 월의 고객 데이터를 집계해서 현재 고객의 전체 모습을 파악</u>

1) 2019년 3월(2019년 3월 31일)에 탈퇴한 고객과 재적 중인 고객을 추출
2) is_deleted 열로 추출

** 2)의 경우 2019년 3월에 탈퇴한 고객은 카운트되지 않기 때문에 주의

In [57]:
customer_merge_df['end_date'] = pd.to_datetime(customer_merge_df['end_date'])
customer_merge_df['end_date']

0      NaT
1      NaT
2      NaT
3      NaT
4      NaT
        ..
4187   NaT
4188   NaT
4189   NaT
4190   NaT
4191   NaT
Name: end_date, Length: 4192, dtype: datetime64[ns]

In [ ]:
# 방법 1)

# >= 20190331 : 전달에 탈퇴 신청한 고객 0331에 탈퇴 '처리'되기 때문
# end_date가 0331면 2월 중 이미 탈퇴 신청 후 탈퇴 고객임
lately_1month_customer = customer_merge_df.loc[(customer_merge_df['end_date'] >= pd.to_datetime('20190331'))
                                               | customer_merge_df['end_date'].isnull()]

print((len(lately_1month_customer))) # 2953명
lately_1month_customer
lately_1month_customer['end_date'].unique() # NaT, '2019-03-31' : 3월에 탈퇴하지 않은 고객

2953


<DatetimeArray>
['NaT', '2019-03-31 00:00:00']
Length: 2, dtype: datetime64[ns]

>> 전체 집계 결과와의 비교

In [ ]:
# 회원 class 구분, 캠페인 구분, 성별, 탈퇴 여부 집계 - 4192
customer_merge_df.groupby(by= 'class_name')['customer_id'].count() # 종일 : 2045, 주간 : 1019, 야간 : 1128
# 전체 회원 중 종일권 비율 : 48.8%

customer_merge_df.groupby(by= 'campaign_name')['customer_id'].count() # 반액 : 650, 무료 : 492, 일반 : 3050
# 전체 회원 중 일반으로 가입한 비율 : 72%

customer_merge_df.groupby(by= 'gender')['customer_id'].count() # 여성 : 1983, 남성 : 2209
# 전체 회원 중 여성 회원 비율 : 47.3%


gender
F    1400
M    1553
Name: customer_id, dtype: int64

In [ ]:
# 3월 유지 회원으로 전체 파악 
# 회원 구분(class), 캠페인 구분, 성별 집계 - 2953

lately_1month_customer.groupby(by= 'class_name')['customer_id'].count() # 종일 : 1444, 주간 : 696, 야간 : 813
# 유지 회원 중 종일권 비율 : 48.9%

lately_1month_customer.groupby(by= 'campaign_name')['customer_id'].count() # 반액 : 311, 무료 : 242, 일반 : 2400
# 전체 회원 중 일반으로 가입한 비율 : 81%
# : 입회 캠페인은 회원 비율 변화에 영향을 미친다고 추측할 수 있습니다.
# : 유지한 회원이 일반으로 가입한 경우가 많음

lately_1month_customer.groupby(by= 'gender')['customer_id'].count() # 여성 : 1400, 남성 : 1553
# 유지 회원 중 여성 회원 비율 : 47.41%

>> 이용 이력 데이터 활용(use_log_df)

- 이용 이력 데이터는 고객 데이터와는 달리 시간적인 요소를 분석할 수 있습니다.                

1. 한 달 이용 횟수의 변화
2. 회원의 (비)정기적인 이용 여부

>> 월 이용 횟수 분석

In [81]:
use_log_df

,log_id,customer_id,usedate,use_month
0,L00000049012330,AS009373,2018-04-01,201804
1,L00000049012331,AS015315,2018-04-01,201804
2,L00000049012332,AS040841,2018-04-01,201804
3,L00000049012333,AS046594,2018-04-01,201804
4,L00000049012334,AS073285,2018-04-01,201804
...,...,...,...,...
197423,L00000049209753,TS977703,2019-03-31,201903
197424,L00000049209754,TS979550,2019-03-31,201903
197425,L00000049209755,TS995299,2019-03-31,201903
197426,L00000049209756,TS995853,2019-03-31,201903


In [ ]:
# 고객별 월 이용 횟수

# use_log_df['usedate'] = pd.to_datetime(use_log_df['usedate'])
use_log_df['usedate']

use_log_df['use_month'] = use_log_df['usedate'].dt.strftime('%Y%m')
use_log_df['use_month']

# 실습)
monthly_use_log_df = pd.pivot_table(data= use_log_df, index= 'customer_id', columns= 'use_month',
                                    aggfunc= 'count')
monthly_use_log_df

# 책)
monthly_use_log_df = use_log_df.groupby(['use_month', 'customer_id'], as_index= False).count()
# as_index = False > use_month를 묶어서 index로 사용하는 것을 하지 않음
monthly_use_log_df
# use_month, customer_id를 기준으로 log_id, use_date에 값이 있으면 count()

monthly_use_log_df.rename(columns= {'log_id' : 'use_count'}, inplace= True)
monthly_use_log_df

monthly_use_log_df.drop('usedate', inplace=  True, axis= 1)
monthly_use_log_df

# 고객 AS002855	: 2018년 4월에 '4'회 이용

,use_month,customer_id,use_count
0,201804,AS002855,4
1,201804,AS009013,2
2,201804,AS009373,3
3,201804,AS015315,6
4,201804,AS015739,7
...,...,...,...
36837,201903,TS995853,8
36838,201903,TS998593,8
36839,201903,TS999079,3
36840,201903,TS999231,6


>> 고객별 평균값, 중앙값, 최댓값, 최솟값 집계

In [ ]:
monthly_use_stats = monthly_use_log_df.groupby(by= 'customer_id')['use_count'].agg(['mean', 'median', 'max', 'min'])
monthly_use_stats = monthly_use_stats.reset_index(drop= False) 
# groupby의 영향으로 customer_id가 index에 들어가 있기 때문에 이를 컬럼으로 변경
# drop= False : 인덱스를 컬럼으로 유지하면서, 정수형 기본 인덱스를 다시 생성
# drop= True : 인덱스를 완전히 제거(컬럼으로 남기지 않음)
monthly_use_stats.head()

# 고객 AS002855 | 평균값 4.5 / 중앙값 5 / 최댓값 7 / 최솟값 2

,customer_id,mean,median,max,min
0,AS002855,4.500000,5.0,7,2
1,AS008805,4.000000,4.0,8,1
2,AS009013,2.000000,2.0,2,2
3,AS009373,5.083333,5.0,7,3
4,AS015233,7.545455,7.0,11,4


>> 정기적 / 비정기적 스포츠 센터 이용 여부

- 정기 이용 플래그

스포츠 센터의 경우 <b><u>지속 요소</u></b> 중 하나로 <b><u>'습관'</u></b>을 생각할 수 있습니다. 여기서는 정기적으로 센터를 이용하는 고객을 특정해 봅시다. <u>정기적이라는 것은 정의</u>하는 방법에 따라 다르지만, 여기서는 <u>매주 같은 요일에 왔는지</u> 아닌지로 판단하겠습니다.                  

>> 고객별 월/요일별로 집계

최댓값이 4 이상인 요일이 하나라도 있는 회원은 플래그를 1로 처리            
<-> 한 주에 한 요일이라도 4회 이상인 회원

In [ ]:
# use_log_df['usedate'] # datetime
use_log_df['use_weekday'] = use_log_df['usedate'].dt.weekday
use_log_df['use_weekday'] # 0 : 월요일 ~ 6 : 일요일

monthly_weekly_use_df = use_log_df.groupby(by= ['customer_id', 'use_month', 'use_weekday'], as_index= False).count()[['customer_id', 'use_month', 'use_weekday', 'log_id']] # 원하는 열만 선택
monthly_weekly_use_df

monthly_weekly_use_df.rename(columns= {'log_id' : 'use_count'}, inplace= True)
# monthly_weekly_use_df.drop(labels= 'usedate', axis= 1, inplace= True) # 원하는 열만 선택하면 삭제할 필요 x
monthly_weekly_use_df

# 회원 AS002855은 2018년 4월 weekday 5에도 4번, 2018년 5월 weekday 5(토)에도 4번 > 매주 토요일에 방문

,customer_id,use_month,use_weekday,use_count
0,AS002855,201804,5,4
1,AS002855,201805,2,1
2,AS002855,201805,5,4
3,AS002855,201806,5,5
4,AS002855,201807,1,1
...,...,...,...,...
93328,TS999855,201901,1,1
93329,TS999855,201901,5,4
93330,TS999855,201901,6,1
93331,TS999855,201902,5,4


>> 고객별 최댓값 계산        

그 최댓값이 4 이상일 경우 플래그 지정

In [138]:
# 월,요일 상관없이 max값
# 이 횟수가 4 이상인 사람은 적어도 어떤 달의 매주 특정 요일에 정기적으로 방문한 고객
monthly_weekly_use_df.info()
use_count_max = monthly_weekly_use_df.groupby(by= ['customer_id'], as_index= False).max()[['customer_id', 'use_count']]
# > customer_id로 groupby.max()하면 use_month, use_weekday, use_count 각 최댓값이 출력됨
# > use_count만 필요하니 해당 열만 가져옴

print(use_count_max)

# 참고)
# use_count가 가장 클 때의 weekday / month를 알고 싶다면!
idx = monthly_weekly_use_df.groupby('customer_id')['use_count'].idxmax()
use_count_max_info = monthly_weekly_use_df.loc[idx]
use_count_max_info


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93333 entries, 0 to 93332
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   customer_id  93333 non-null  object
 1   use_month    93333 non-null  object
 2   use_weekday  93333 non-null  int32 
 3   use_count    93333 non-null  int64 
dtypes: int32(1), int64(1), object(2)
memory usage: 2.5+ MB
     customer_id  use_count
0       AS002855          5
1       AS008805          4
2       AS009013          2
3       AS009373          5
4       AS015233          5
...          ...        ...
4187    TS995853          5
4188    TS998593          5
4189    TS999079          5
4190    TS999231          5
4191    TS999855          5

[4192 rows x 2 columns]


,customer_id,use_month,use_weekday,use_count
3,AS002855,201806,5,5
28,AS008805,201806,5,4
50,AS009013,201804,0,2
55,AS009373,201807,6,5
94,AS015233,201809,6,5
...,...,...,...,...
93238,TS995853,201903,6,5
93242,TS998593,201809,5,5
93267,TS999079,201804,6,5
93300,TS999231,201810,1,5
